In [1]:
import scanpy as sc

data_dir = '../../dataset/merfish_human_neocortex_SSL.h5ad'
adata = sc.read_h5ad(data_dir)
adata.X = adata.raw.X.copy()
print(adata.X)
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
adata.raw.X

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 12532157 stored elements and shape (234125, 300)>
  Coords	Values
  (0, 9)	5
  (0, 16)	2
  (0, 18)	3
  (0, 23)	1
  (0, 24)	1
  (0, 25)	5
  (0, 27)	1
  (0, 28)	1
  (0, 29)	6
  (0, 30)	1
  (0, 31)	1
  (0, 34)	6
  (0, 37)	1
  (0, 45)	21
  (0, 46)	1
  (0, 53)	1
  (0, 58)	1
  (0, 60)	1
  (0, 61)	1
  (0, 67)	2
  (0, 68)	1
  (0, 74)	1
  (0, 81)	1
  (0, 86)	1
  (0, 87)	2
  :	:
  (234123, 261)	1
  (234124, 14)	2
  (234124, 18)	1
  (234124, 25)	3
  (234124, 55)	1
  (234124, 58)	1
  (234124, 62)	1
  (234124, 66)	1
  (234124, 75)	1
  (234124, 89)	1
  (234124, 104)	1
  (234124, 106)	1
  (234124, 107)	1
  (234124, 117)	2
  (234124, 126)	1
  (234124, 146)	3
  (234124, 171)	1
  (234124, 184)	1
  (234124, 200)	1
  (234124, 215)	3
  (234124, 221)	1
  (234124, 237)	1
  (234124, 251)	1
  (234124, 261)	1
  (234124, 297)	1


<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 12532157 stored elements and shape (234125, 300)>

In [2]:
adata

AnnData object with n_obs × n_vars = 234125 × 300
    obs: 'Cell_ID', 'orig.ident', 'nCount_Vizgen', 'nFeature_Vizgen', 'z', 'volume', 'fov', 'nCount_SCT', 'nFeature_SCT', 'seurat_clusters', 'Sample_ID', 'Estimated_postconceptional_age_in_days', 'Group', 'Region', 'Region_details', 'integrated_snn_res.2.8', 'class', 'subclass', 'type', 'nCount_niche', 'nFeature_niche', 'niches', 'niche_name', 'type_updated', 'x', 'y', 'cell_type', 'tissue', 'slice_id', 'cellcharter_scVI', 'cellcharter_SSL_BT_ZS', 'cellcharter_SSL_RM_ZS', 'STAGATE_SSL_BT_ZS', 'STAGATE_SSL_RM_ZS', 'STAGATE_original'
    var: 'index_', 'features'
    uns: 'BT_FT_predictions_42', 'BT_FT_target_names_42', 'BT_FT_train_loss_42', 'BT_FT_val_loss_42', 'BT_FT_y_test_42', 'log1p'
    obsm: 'SSL_BT_FT', 'SSL_BT_FT_42', 'SSL_BT_ZS', 'SSL_GP_FT', 'SSL_GP_ZS', 'SSL_RM_FT', 'SSL_RM_ZS', 'STAGATE_SSL_BT_ZS', 'STAGATE_SSL_RM_ZS', 'STAGATE_original', 'cellcharter_SSL_BT_ZS', 'cellcharter_SSL_RM_ZS', 'cellcharter_scVI', 'scVI', 'spatial'

In [3]:
%load_ext autoreload
%autoreload 2
import torch
import lightning.pytorch as pl
from self_supervision.models.lightning_modules.cellnet_autoencoder import MLPBarlowTwins
from self_supervision.estimator.cellnet import EstimatorAutoEncoder

# 设置你的 .ckpt 文件路径
ckpt_path = "../../sc_pretrained/Pretrained Models/BarlowTwins.ckpt"

# 模型参数
units_encoder = [512, 512, 256, 256, 64]

# 初始化 EstimatorAutoEncoder 实例
estim = EstimatorAutoEncoder(data_path=None)  # 如果没有实际数据路径，可以设置为None

# 加载预训练模型
estim.model = MLPBarlowTwins(
        gene_dim=19331,  # 根据你的数据调整
        batch_size=2048,  # 根据你的需要调整
        units_encoder=units_encoder,
        CHECKPOINT_PATH=ckpt_path
    )


estim.trainer = pl.Trainer(accelerator="gpu", devices=1 if torch.cuda.is_available() else None)
estim.model

/home/hanchuangyi/miniconda3/envs/ssl/lib/python3.10/site-packages/merlin/dtypes/mappings/tf.py:52: UserWarning: Tensorflow dtype mappings did not load successfully due to an error: No module named 'tensorflow'
  warn(f"Tensorflow dtype mappings did not load successfully due to an error: {exc.msg}")
/home/hanchuangyi/miniconda3/envs/ssl/lib/python3.10/site-packages/merlin/dtypes/mappings/triton.py:53: UserWarning: Triton dtype mappings did not load successfully due to an error: No module named 'tritonclient'
  warn(f"Triton dtype mappings did not load successfully due to an error: {exc.msg}")


GPU available: True (cuda), used: True


TPU available: False, using: 0 TPU cores


HPU available: False, using: 0 HPUs


MLPBarlowTwins(
  (train_metrics): MetricCollection(
    (explained_var_uniform): ExplainedVariance()
    (explained_var_weighted): ExplainedVariance()
    (mse): MeanSquaredError(),
    prefix=train_
  )
  (val_metrics): MetricCollection(
    (explained_var_uniform): ExplainedVariance()
    (explained_var_weighted): ExplainedVariance()
    (mse): MeanSquaredError(),
    prefix=val_
  )
  (test_metrics): MetricCollection(
    (explained_var_uniform): ExplainedVariance()
    (explained_var_weighted): ExplainedVariance()
    (mse): MeanSquaredError(),
    prefix=test_
  )
  (inner_model): MLP(
    (0): Linear(in_features=19331, out_features=512, bias=True)
    (1): SELU()
    (2): Dropout(p=0.0, inplace=False)
    (3): Linear(in_features=512, out_features=512, bias=True)
    (4): SELU()
    (5): Dropout(p=0.0, inplace=False)
    (6): Linear(in_features=512, out_features=256, bias=True)
    (7): SELU()
    (8): Dropout(p=0.0, inplace=False)
    (9): Linear(in_features=256, out_features=25

In [4]:
# 加载 checkpoint
checkpoint = torch.load(ckpt_path)
estim.model.inner_model.load_state_dict({k.replace('backbone.', ''): v for k, v in checkpoint.items() if 'backbone' in k})

# 设定模型为评估模式
estim.model.eval()

MLPBarlowTwins(
  (train_metrics): MetricCollection(
    (explained_var_uniform): ExplainedVariance()
    (explained_var_weighted): ExplainedVariance()
    (mse): MeanSquaredError(),
    prefix=train_
  )
  (val_metrics): MetricCollection(
    (explained_var_uniform): ExplainedVariance()
    (explained_var_weighted): ExplainedVariance()
    (mse): MeanSquaredError(),
    prefix=val_
  )
  (test_metrics): MetricCollection(
    (explained_var_uniform): ExplainedVariance()
    (explained_var_weighted): ExplainedVariance()
    (mse): MeanSquaredError(),
    prefix=test_
  )
  (inner_model): MLP(
    (0): Linear(in_features=19331, out_features=512, bias=True)
    (1): SELU()
    (2): Dropout(p=0.0, inplace=False)
    (3): Linear(in_features=512, out_features=512, bias=True)
    (4): SELU()
    (5): Dropout(p=0.0, inplace=False)
    (6): Linear(in_features=512, out_features=256, bias=True)
    (7): SELU()
    (8): Dropout(p=0.0, inplace=False)
    (9): Linear(in_features=256, out_features=25

In [5]:
import pandas as pd
var_df = pd.read_parquet('../../sc_pretrained/var.parquet')
var_df

,feature_id,feature_name
0,ENSG00000186092,OR4F5
1,ENSG00000284733,OR4F29
2,ENSG00000284662,OR4F16
3,ENSG00000187634,SAMD11
4,ENSG00000188976,NOC2L
...,...,...
19326,ENSG00000288702,UGT1A3
19327,ENSG00000288705,UGT1A5
19328,ENSG00000182484,WASH6P
19329,ENSG00000288622,PDCD6-AHRR


In [6]:
all_genes = var_df['feature_name'].tolist()
all_genes

['OR4F5',
 'OR4F29',
 'OR4F16',
 'SAMD11',
 'NOC2L',
 'KLHL17',
 'PLEKHN1',
 'PERM1',
 'HES4',
 'ISG15',
 'AGRN',
 'RNF223',
 'C1orf159',
 'TTLL10',
 'TNFRSF18',
 'TNFRSF4',
 'SDF4',
 'B3GALT6',
 'C1QTNF12',
 'UBE2J2',
 'SCNN1D',
 'ACAP3',
 'PUSL1',
 'INTS11',
 'CPTP',
 'TAS1R3',
 'DVL1',
 'MXRA8',
 'AURKAIP1',
 'CCNL2',
 'MRPL20',
 'ANKRD65',
 'TMEM88B',
 'VWA1',
 'ATAD3C',
 'ATAD3B',
 'ATAD3A',
 'TMEM240',
 'SSU72',
 'FNDC10',
 'MIB2',
 'MMP23B',
 'CDK11B',
 'SLC35E2B',
 'CDK11A',
 'NADK',
 'GNB1',
 'CALML6',
 'TMEM52',
 'CFAP74',
 'GABRD',
 'PRKCZ',
 'FAAP20',
 'SKI',
 'MORN1',
 'RER1',
 'PEX10',
 'PLCH2',
 'PANK4',
 'HES5',
 'TNFRSF14',
 'PRXL2B',
 'MMEL1',
 'TTC34',
 'ACTRT2',
 'PRDM16',
 'ARHGEF16',
 'MEGF6',
 'TPRG1L',
 'WRAP73',
 'TP73',
 'CCDC27',
 'SMIM1',
 'LRRC47',
 'CEP104',
 'DFFB',
 'C1orf174',
 'AJAP1',
 'NPHP4',
 'KCNAB2',
 'CHD5',
 'RPL22',
 'RNF207',
 'ICMT',
 'HES3',
 'GPR153',
 'ACOT7',
 'HES2',
 'ESPN',
 'TNFRSF25',
 'PLEKHG5',
 'NOL9',
 'TAS1R1',
 'ZBTB48',
 'KLH

In [7]:
adata.var['gene_name']=adata.var.index
adata.var['gene_name']

features
TFAP2C          TFAP2C
HGF                HGF
NAMPT            NAMPT
VIM                VIM
CD38              CD38
               ...    
LINC01170    LINC01170
SATB1-AS1    SATB1-AS1
CFAP54          CFAP54
RMST              RMST
NOTCH1          NOTCH1
Name: gene_name, Length: 300, dtype: object

In [8]:
import numpy as np
# 初始化一个新的数据矩阵，形状为 (adata.X.shape[0], len(all_genes))，填充为零
new_data = np.zeros((adata.X.shape[0], len(all_genes)), dtype=np.float32)


In [9]:
existing_genes = adata.var['gene_name']
existing_genes

features
TFAP2C          TFAP2C
HGF                HGF
NAMPT            NAMPT
VIM                VIM
CD38              CD38
               ...    
LINC01170    LINC01170
SATB1-AS1    SATB1-AS1
CFAP54          CFAP54
RMST              RMST
NOTCH1          NOTCH1
Name: gene_name, Length: 300, dtype: object

In [10]:
# 将所有基因名称转换为小写
all_genes_lower = [gene.lower() for gene in all_genes]
adata_genes_lower = [gene.lower() for gene in existing_genes]

# 将两个列表转换为集合
all_genes_set = set(all_genes_lower)
adata_genes_set = set(adata_genes_lower)

# 计算交集
matching_genes = all_genes_set.intersection(adata_genes_set)
matching_count = len(matching_genes)
# 计算不匹配的基因
non_matching_genes = adata_genes_set - matching_genes
non_matching_count = len(non_matching_genes)


# 输出结果
print(f"匹配的基因数量: {matching_count}")
print(f"匹配的基因列表: {matching_genes}")
non_matching_genes


匹配的基因数量: 281
匹配的基因列表: {'bmper', 'grm3', 'lhfpl3', 'atp10b', 'lama2', 'zic1', 'mob3b', 'sst', 'nefl', 'flt1', 'kiaa1217', 'pou6f2', 'grin2a', 'il1rapl2', 'gad2', 'spp1', 'cd38', 'ebf3', 'itih5', 'spata17', 'clmp', 'runx1', 'kcns3', 'fezf2', 'adamtsl1', 'neurod1', 'lipg', 'kit', 'vwc2l', 'col12a1', 'mx1', 'kif11', 'opcml', 'ltbp1', 'klhl1', 'camk2a', 'reln', 'ccdc175', 'dach1', 'daam2', 'rit2', 'enpp2', 'bcl11b', 'plp1', 'adarb2', 'cdh18', 'prss12', 'tfrc', 'pou3f2', 'spock3', 'grin3a', 'rnf220', 'abca8', 'igfbp7', 'pcbp3', 'igfbp5', 'cd44', 'spata6', 'tfpi', 'mdga1', 'cenpf', 'satb2', 'epha3', 'arx', 'pcdh11y', 'tcf7l2', 'palmd', 'sema5a', 'kcnq5', 'ccser1', 'ptchd4', 'thsd4', 'fam160a1', 'nhlh2', 'ugt8', 'runx1t1', 'nkain2', 'cebpd', 'ifit3', 'brip1', 'znf536', 'nxph2', 'cadps2', 'nxph1', 'col1a2', 'cntnap4', 'gli3', 'sp8', 'olfm3', 'tnc', 'tfap2c', 'eogt', 'hcrtr2', 'znf804b', 'nampt', 'fyb2', 'isg15', 'zbbx', 'cobll1', 'trhde', 'col24a1', 'armc3', 'fos', 'ablim1', 'glra3', 'pcsk6', '

{'dlx6-as1',
 'fam106a',
 'kcnq1ot1',
 'lhx1-dt',
 'linc00609',
 'linc01170',
 'linc01619',
 'linc01965',
 'linc02147',
 'linc02487',
 'lncarod',
 'neat1',
 'nfia-as2',
 'nr2f2-as1',
 'obi1-as1',
 'rmst',
 'satb1-as1',
 'sgo1-as1',
 'tctex1d1'}

In [11]:
gene_to_index = {gene: idx for idx, gene in enumerate(all_genes_lower)}
gene_to_index

{'or4f5': 0,
 'or4f29': 1,
 'or4f16': 2,
 'samd11': 3,
 'noc2l': 4,
 'klhl17': 5,
 'plekhn1': 6,
 'perm1': 7,
 'hes4': 8,
 'isg15': 9,
 'agrn': 10,
 'rnf223': 11,
 'c1orf159': 12,
 'ttll10': 13,
 'tnfrsf18': 14,
 'tnfrsf4': 15,
 'sdf4': 16,
 'b3galt6': 17,
 'c1qtnf12': 18,
 'ube2j2': 19,
 'scnn1d': 20,
 'acap3': 21,
 'pusl1': 22,
 'ints11': 23,
 'cptp': 24,
 'tas1r3': 25,
 'dvl1': 26,
 'mxra8': 27,
 'aurkaip1': 28,
 'ccnl2': 29,
 'mrpl20': 30,
 'ankrd65': 31,
 'tmem88b': 32,
 'vwa1': 33,
 'atad3c': 34,
 'atad3b': 35,
 'atad3a': 36,
 'tmem240': 37,
 'ssu72': 38,
 'fndc10': 39,
 'mib2': 40,
 'mmp23b': 41,
 'cdk11b': 42,
 'slc35e2b': 43,
 'cdk11a': 44,
 'nadk': 45,
 'gnb1': 46,
 'calml6': 47,
 'tmem52': 48,
 'cfap74': 49,
 'gabrd': 50,
 'prkcz': 51,
 'faap20': 52,
 'ski': 53,
 'morn1': 54,
 'rer1': 55,
 'pex10': 56,
 'plch2': 57,
 'pank4': 58,
 'hes5': 59,
 'tnfrsf14': 60,
 'prxl2b': 61,
 'mmel1': 62,
 'ttc34': 63,
 'actrt2': 64,
 'prdm16': 65,
 'arhgef16': 66,
 'megf6': 67,
 'tprg1l': 68

In [12]:
only_in_all_genes = all_genes_set - adata_genes_set

only_in_adata_genes = adata_genes_set - all_genes_set

# 输出结果
print(f"仅在 all_genes 中存在的基因数量: {len(only_in_all_genes)}")
print(f"仅在 all_genes 中存在的基因: {only_in_all_genes}")

print(f"仅在 adata_genes 中存在的基因数量: {len(only_in_adata_genes)}")
print(f"仅在 adata_genes 中存在的基因: {only_in_adata_genes}")


仅在 all_genes 中存在的基因数量: 19050
仅在 all_genes 中存在的基因: {'eef1akmt2', 'flnc', 'ctsf', 'rimklb', 'or1s1', 'gclm', 'sdad1', 'mtf2', 'znf99', 'gys1', 'krt75', 'syp', 'garnl3', 'ralgds', 'pcdhgb3', 'efna5', 'cep57', 'vegfb', 'c18orf63', 'lingo3', 'krt38', 'arpc5l', 'ralgapa1', 'ppm1b', 'pard3b', 'spdye1', 'cebpg', 'fads1', 'man2a1', 'serpinf2', 'cacna2d2', 'rad1', 'fam117b', 'mapk1', 'klhdc7b', 'tnip1', 'spdye5', 'fam90a16p', 'prkd1', 'gsta4', 'mpo', 'dhrs9', 'fars2', 'stambpl1', 'chst8', 'tmem9', 'ccdc22', 'ankrd53', 'ninl', 'cenpvl2', 'mln', 'ddo', 'grsf1', 'c17orf113', 'smim26', 'h2ab1', 'las1l', 'plekhg6', 'mrgprg', 'c10orf105', 'nxnl2', 'f10', 'tnfrsf9', 'epor', 'nek8', 'tmem216', 'wdr72', 'rlim', 'pld1', 'clvs2', 'plekhd1', 'fam229a', 'h3c2', 'srfbp1', 'cdrt15l2', 'fetub', 'pum2', 'scygr6', 'dynlt2b', 'klhl21', 'c17orf75', 'gldc', 'tyms', 'eda2r', 'tmub2', 'angel2', 'tbl1x', 'ccser2', 'tspan8', 'c16orf72', 'h2bw1', 'cox6b1', 'ccdc96', 'znf391', 'slc2a7', 'prdx5', 'prss23', 'adamts16', 'mxd

In [13]:
dense_adata_X = adata.X.toarray()
for i, gene in enumerate(adata_genes_lower):
    if gene in gene_to_index:
        new_data[:, gene_to_index[gene]] = dense_adata_X[:, i]
    else:
        print(f'Gene {gene} not found in all_genes list')

Gene tctex1d1 not found in all_genes list


Gene linc02487 not found in all_genes list
Gene nfia-as2 not found in all_genes list
Gene lncarod not found in all_genes list
Gene dlx6-as1 not found in all_genes list
Gene sgo1-as1 not found in all_genes list


Gene linc02147 not found in all_genes list
Gene linc01965 not found in all_genes list
Gene linc00609 not found in all_genes list
Gene linc01619 not found in all_genes list
Gene kcnq1ot1 not found in all_genes list
Gene obi1-as1 not found in all_genes list
Gene fam106a not found in all_genes list
Gene neat1 not found in all_genes list
Gene nr2f2-as1 not found in all_genes list
Gene lhx1-dt not found in all_genes list
Gene linc01170 not found in all_genes list
Gene satb1-as1 not found in all_genes list
Gene rmst not found in all_genes list


In [14]:
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.model_selection import train_test_split


label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(adata.obs['cell_type'])  # 预先编码标签


random_seed = 42
X_train_val, X_test, y_train_val, y_test = train_test_split(
    new_data, labels_encoded, test_size=0.15, random_state=random_seed)


X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.1765, random_state=random_seed)  # 0.1765 是为了让验证集占 15%

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
estim.model.to(device)
estim.model.eval()
with torch.no_grad():
    X_train_tensor = torch.tensor(X_train).float().to(device)
    X_test_tensor = torch.tensor(X_test).float().to(device)
    train_embeddings = estim.model.inner_model(X_train_tensor).detach().cpu().numpy()
    test_embeddings = estim.model.inner_model(X_test_tensor).detach().cpu().numpy()


cuda


In [15]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report

    

    # 初始化和训练KNN分类器
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(train_embeddings, y_train)
    
    # 模型预测
predictions = knn.predict(test_embeddings)

    # 计算准确率和 F1 分数
accuracy = accuracy_score(y_test, predictions)
print(f"KNN Accuracy on Test Data: {accuracy}")
f1 = f1_score(y_test, predictions, average='weighted')
print(f"Weighted F1 Score: {f1}")
    
macro_f1 = f1_score(y_test, predictions, average='macro')
print(f'Macro F1 Score: {macro_f1}')

    # 计算随机猜测的准确率
class_probabilities = np.bincount(y_test) / len(y_test)
random_accuracy = np.sum(class_probabilities ** 2)
print(f"Random Guess Accuracy: {random_accuracy}")

    # 生成分类报告
report = classification_report(y_test, predictions, target_names=label_encoder.classes_)
print(report)

KNN Accuracy on Test Data: 0.6977989122697116
Weighted F1 Score: 0.6971440909822255
Macro F1 Score: 0.6828487698106848
Random Guess Accuracy: 0.05408382924188506
                          precision    recall  f1-score   support

       Astrocyte-Fibrous       0.75      0.84      0.79      2056
  Astrocyte-Protoplasmic       0.72      0.73      0.73      1958
          EN-IT-Immature       0.60      0.69      0.64      3827
              EN-L2_3-IT       0.58      0.68      0.63      2843
                EN-L4-IT       0.63      0.59      0.61      2585
                EN-L5-ET       0.58      0.54      0.56        98
                EN-L5-IT       0.68      0.67      0.68      1670
              EN-L5_6-NP       0.83      0.69      0.75       344
                EN-L6-CT       0.68      0.71      0.70       778
                EN-L6-IT       0.74      0.67      0.70      1914
                  EN-L6b       0.78      0.61      0.68      1218
              EN-Newborn       0.73      0.61

In [16]:
with torch.no_grad():
    new_data_tensor = torch.tensor(new_data).float().to(device)
    SSL_embeddings = estim.model.inner_model(new_data_tensor).detach().cpu().numpy()
SSL_embeddings

array([[ 0.6503338 , -1.6183035 ,  1.5518377 , ...,  4.919082  ,
        -2.9327168 ,  2.7836118 ],
       [ 1.6452467 , -2.2684464 ,  0.4458326 , ...,  3.428311  ,
        -1.8959248 ,  3.1119132 ],
       [ 1.1750375 , -1.6273767 ,  1.0605824 , ...,  4.0755396 ,
        -1.8092277 ,  2.866513  ],
       ...,
       [ 1.6311089 , -1.8494682 ,  2.0660691 , ...,  4.7446613 ,
        -3.5055978 ,  1.7167863 ],
       [ 1.2391025 , -2.009054  ,  1.5966071 , ...,  5.059773  ,
        -2.8023703 ,  2.9689407 ],
       [ 1.3355403 , -3.9589353 ,  0.54607266, ...,  3.81955   ,
        -3.0939088 ,  2.844739  ]], dtype=float32)

In [17]:
with torch.no_grad():
    new_data_tensor = torch.tensor(new_data).float().to(device)
    SSL_embeddings = estim.model.inner_model(new_data_tensor).detach().cpu().numpy()
new_adata = sc.read_h5ad(data_dir)
new_adata.obsm[f'SSL_BT_ZS_{random_seed}'] = SSL_embeddings
new_adata.uns[f'BT_ZS_y_test_{random_seed}'] = y_test
new_adata.uns[f'BT_ZS_predictions_{random_seed}'] = predictions
new_adata.uns[f'BT_ZS_target_names_{random_seed}'] = label_encoder.classes_
new_adata.write_h5ad(data_dir)

In [18]:

import pandas as pd
import os
import re

# 当前 Notebook 文件名
notebook_name = "MERSCOPE_human_neocortex_barlow_twins_zero_shot_42.ipynb"

# 初始化需要打印的值
init_train_loss = train_losses[0] if 'train_losses' in globals() else None
init_val_loss = val_losses[0] if 'val_losses' in globals() else None
converged_epoch = len(train_losses) - patience if 'train_losses' in globals() else None
converged_val_loss = best_val_loss if 'best_val_loss' in globals() else None

# 打印所有所需的指标
print("Metrics Summary:")
if 'train_losses' in globals():
    print(f"init_train_loss\tinit_val_loss\tconverged_epoch\tconverged_val_loss\tmacro_f1\tweighted_f1\tmicor_f1")
    print(f"{init_train_loss:.3f}\t{init_val_loss:.3f}\t{converged_epoch}\t{converged_val_loss:.3f}\t{macro_f1:.3f}\t{f1:.3f}\t{accuracy:.3f}")
else:
    print(f"macro_f1\tweighted_f1\tmicor_f1")
    print(f"{macro_f1:.3f}\t{f1:.3f}\t{accuracy:.3f}")

# 保存结果到 CSV 文件
output_data = {
    'dataset_split_random_seed': [int(random_seed)],
    'dataset': ['MERSCOPE_human_neocortex'],
    'method': [re.search(r'neocortex_(.*?)_\d+', notebook_name).group(1)],
    'init_train_loss': [init_train_loss if init_train_loss is not None else ''],
    'init_val_loss': [init_val_loss if init_val_loss is not None else ''],
    'converged_epoch': [converged_epoch if converged_epoch is not None else ''],
    'converged_val_loss': [converged_val_loss if converged_val_loss is not None else ''],
    'macro_f1': [macro_f1],
    'weighted_f1': [f1],
    'micor_f1': [accuracy]
}
output_df = pd.DataFrame(output_data)

# 保存到当前目录下名为 results 的文件夹中
if not os.path.exists('results'):
    os.makedirs('results')

csv_filename = f"results/{os.path.splitext(notebook_name)[0]}_results.csv"
output_df.to_csv(csv_filename, index=False)


Metrics Summary:
macro_f1	weighted_f1	micor_f1
0.683	0.697	0.698
